In [25]:
import pandas as pd
import numpy as np
import vaex
import pickle

In [12]:
traf = vaex.open('../../TRAFICO/traf_hist_unir.hdf5')

traf.info()

In [26]:
acci_filtered_vaex = vaex.open('../../acci_filtered_vaex.hdf5')
accis_id = acci_filtered_vaex.to_pandas_df()

ids = accis_id.idelem.value_counts().index

In [27]:
ids

Index(['4206', '4436', '4774', '3901', '4452', '5828', '4170', '3798', '5484',
       '4166',
       ...
       '5217', '6608', '5388', '6266', '6420', '10443', '10406', '10584',
       '10462', '5426'],
      dtype='object', length=4371)

In [28]:
acci = pd.read_csv('../data/biblio_acci.csv')

In [29]:
import tqdm

for id in tqdm.tqdm(ids):

    #filtrar por idelem

    traf = traf[traf.idelem == id]

    #generar tabla para modelo

    if len(traf) > 0:

        traf = traf.to_pandas_df()

        traf_acci = pd.merge(traf, acci, on='unir', how='left')

        traf_acci['accidente'] = traf_acci['accidente'].fillna(0)

        traf_acci.drop(['unir', 'idelem'], axis=1, inplace=True)

        traf_acci['fecha'] = traf_acci['fecha_full'].str.split(' ').str[0]

        traf_acci['hora'] = traf_acci['fecha_full'].str.split(' ').str[1]

        traf_acci.drop('fecha_full', axis=1, inplace=True)

        traf_acci['prec'] = pd.to_numeric(traf_acci['prec'].str.replace(',', '.'), errors='coerce')

        traf_acci['velmedia'] = pd.to_numeric(traf_acci['velmedia'].str.replace(',', '.'), errors='coerce')

        traf_acci.fillna(0, inplace=True)

        traf_acci['mes'] = traf_acci['fecha'].str.split('-').str[1]

        traf_acci['fecha'] = pd.to_datetime(traf_acci['fecha'])

        traf_acci['dia_semana'] = traf_acci['fecha'].dt.dayofweek

        traf_acci.drop('fecha', axis=1, inplace=True)

        traf_acci['dia_semana'] = traf_acci['dia_semana'].astype('object')

        #get dummies

        traf_acci_dum = pd.get_dummies(traf_acci, columns=['mes', 'dia_semana', 'hora'])

        #Xtest, ytest

        from sklearn.model_selection import train_test_split

        X = traf_acci_dum.drop('accidente', axis=1)

        y = traf_acci_dum['accidente']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        #scale

        from sklearn.preprocessing import StandardScaler

        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        #model

        model = LogisticRegression()
        model.fit(X_train, y_train)

        filename = 'modelo_' + id + '.sav'
        pickle.dump(model, open('../modelos/' + filename, 'wb'))
    

  0%|          | 0/4371 [02:15<?, ?it/s]

KeyboardInterrupt

